In [16]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model
from deeppavlov import configs

import umap
import hdbscan

import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

from pathlib import Path
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
import plotly.express as px

In [17]:
path_corpus_tut = Path('../data/corpora/tutby_126784.csv')

# path_model = Path('../data/model/bert/sentence_ru_cased_L-12_H-768_A-12_pt')
path_model = Path('../data/model/bert/rubert_cased_L-12_H-768_A-12_pt')

path_emb = Path('../data/emb/tutby_126784_rubert.npy')

path_plot = Path('../data/plot/tutby_126784_rubert.html')

In [3]:
data = pd.read_csv(path_corpus_tut)

# corpus = data['document'].fillna('')
corpus = data['header'].tolist()

print(len(corpus))
display(data.head(3))

126784


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,['министерства']


In [4]:
%%time

# bert_config = read_json(configs.embedder.bert_embedder)
# bert_config['metadata']['variables']['BERT_PATH'] = path_model

# model = build_model(bert_config)

Wall time: 0 ns


In [5]:
%%time

# batch_size = 256
# embeddings = []

# for i in tqdm(range(len(corpus) // batch_size + 1)):
#     batch = corpus[batch_size * i : batch_size * (i + 1)]

#     sent_mean_embs = model(batch)[5]
#     embeddings += [sent_mean_embs]

# embeddings = np.concatenate(embeddings)


# with open(path_emb, 'wb') as file:
#     np.save(file, embeddings)

with open(path_emb, 'rb') as file:
    embeddings = np.load(file)

    
print(embeddings.shape)

(126784, 768)
Wall time: 272 ms


In [8]:
for k in range(20):
    d = cdist(embeddings[k:k+1], embeddings, metric='cosine')[0]
    m = d[k+1:].argmin()
    print(k)
    print(corpus[k])
    print(corpus[m + k+1])

In [6]:
%%time

# umap_model = umap.UMAP(n_neighbors=15, n_components=5, metric='cosine').fit(embeddings)

Wall time: 1min 50s


In [7]:
%%time

# cluster = hdbscan.HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom').fit(umap_model.embedding_)

# labels = cluster.labels_

Wall time: 10.3 s


In [13]:
%%time

umap_model2 = umap.UMAP(n_neighbors=15, n_components=2, metric='cosine').fit(embeddings)

In [18]:
%%time

hover_data = {
    'header': data['header'], 
    'label': data['label'], 
    'tags': data['tags'], 
}

points = umap_model2.embedding_

fig = px.scatter(
    x=points[:, 0], 
    y=points[:, 1],
    hover_data=hover_data, 
    width=1000, 
    height=1000,
)
fig.update_traces(marker=dict(size=4))

fig.write_html(str(path_plot))

Wall time: 8.17 s
